In [6]:
import os
import json
import pandas
import numpy as np
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.externals import joblib
% matplotlib inline

In [57]:
Ned = []
Med = []
NRd = []
MRd = []
fck = []
As = []
bh = []
a = []
xu = []
l0 = []
listdir = os.listdir("../../data_set/buckling")


for file in listdir:
    f = open("../../data_set/buckling/" + file)
    d = json.loads(f.read())
    f.close()

    for o in d:
        Ned.append(o["Ned"])
        Med.append(o["Med"])
        NRd.append(o["NRd"])
        MRd.append(o["MRd"])
        fck.append(o["fck"])
        As.append(o["As"])
        bh.append(o["bh"])
        a.append(o["a"])
        xu.append(o["xu"])
        l0.append(o["l0"])

rho = []
for i in range(len(As)):
    rho.append(As[i] / bh[i]**2)
df = pandas.DataFrame({
        "l0": l0,
        "Ned": Ned,
        "MRd": MRd,
        "fck": fck,
        "bh": bh,
        "rho": rho,
    }
    )
print(df.shape)
df = df[df["fck"] == 30]
print(df.shape)
df.head(5)



(9766650, 6)
(3327000, 6)
0.04


In [33]:
# Scale all features by their mean and std-dev to value 0 - 1. This is the correct one.

feat_df = df[["MRd", "rho", "Ned", "l0"]]
res_df = df[["bh"]]

scale_feat = preprocessing.MinMaxScaler()
scale_feat.fit(feat_df)
print(scale_feat.transform(feat_df))


[[ 0.          0.1088725   0.          0.34693878]
 [ 0.0043363   0.0877827   0.          0.34693878]
 [ 0.01404371  0.0858067   0.          0.34693878]
 ..., 
 [ 0.531571    0.03070257  0.31010453  0.79591837]
 [ 0.62073354  0.03911252  0.31010453  0.79591837]
 [ 0.71418745  0.04636459  0.31010453  0.79591837]]


In [8]:
with open("scale_fck30.pkl", "wb") as f:
    pickle.dump(scale_feat, f)

In [9]:
m = joblib.load("buckling_fck30")

In [110]:
input = {
    "Med": [10e6, 0],
    "rho": [0.000833333333333, 0],
    "Ned": [180e3, 0],
    "l0": [12e3, 0],
}

inp = pandas.DataFrame(input)
print(inp.head())

X = scale_feat.transform(inp[["Med", "rho", "Ned", "l0"]])

print("\n",X)

print("\nPrediction:", m.predict(X))

          Med       Ned       l0       rho
0  10000000.0  180000.0  12000.0  0.000833
1         0.0       0.0      0.0  0.000000

 [[-0.02845805  0.00845188  0.02787456  1.02040816]
 [-0.03846104 -0.01264489 -0.03484321 -0.20408163]]

Prediction: [ 303.30655504  266.35765747]


array([[ 0.26162867,  0.49367756,  0.31358885,  0.40816327],
       [-0.03846104, -0.01264489, -0.03484321, -0.20408163]])

array([ 321.72197025,  266.35765747])

In [28]:
joblib.dump(m, 'buckling_fck30.pkl') 

['buckling_fck30.pkl']